In [32]:
from datetime import datetime
import plotly.graph_objects as pg
import plotly.express as px
import plotly.io as pio
import pandas as pd
import locale
locale.setlocale(locale.LC_TIME, "")


from cashd import db

CORES = ["#478eff", "gray"]

Este notebook será usado para experimentar e desenvolver visualizações de maneira mais eficiente.

In [37]:
periodo = "sem"
n = 10

tbl = db.saldos_transac_periodo(periodo=periodo, n=n)
tbl["Data"].apply(lambda x: datetime.strptime(x+"-0", "%Y-%W-%w"))

0    2022-11-27
1    2022-12-11
2    2023-01-08
3    2023-02-05
4    2023-03-05
        ...    
58   2024-04-28
59   2024-05-05
60   2024-05-12
61   2024-05-19
62   2024-05-26
Name: Data, Length: 63, dtype: datetime64[ns]

In [40]:
### balancos_por_periodo
periodo = "sem"
n = 10

tbl = db.saldos_transac_periodo(periodo=periodo, n=n)
if periodo == "sem":
    tbl["Data"] = tbl["Data"].apply(lambda x: datetime.strptime(x+"-0", "%Y-%W-%w"))
tbl["Data"] = pd.to_datetime(tbl.Data)
tbl["SomasDisplay"] = tbl["Somas"]\
    .apply(lambda x: f"{x:_.2f}".replace(".", ",").replace("_", " "))
tbl["AbatDisplay"] = tbl["Abatimentos"]\
    .apply(lambda x: f"{x:_.2f}".replace(".", ",").replace("_", " "))


layout = pg.Layout(
    margin=dict(l=0, r=0, t=0, b=0),
    template="none",
    showlegend=False,
    hovermode="x unified",
    xaxis=dict(
        tickmode="array",
        tickvals=[i for i in tbl["Data"]],
        ticktext=[i.strftime("%B de %Y") for i in tbl["Data"]]
    )
)

fig = pg.Figure(layout=layout)
fig.add_trace(pg.Bar(
    x=tbl["Data"], y=tbl["Somas"], name="Somas",
    customdata=tbl[["SomasDisplay"]],
    hovertemplate="<b>R$ %{customdata[0]}</b>",
    offsetgroup=0,
    marker=dict(color=CORES[0])
))
fig.add_trace(pg.Bar(
    x=tbl["Data"], y=tbl["Abatimentos"], name="Abatimentos",
    customdata=tbl[["AbatDisplay"]],
    hovertemplate="<b>R$ %{customdata[0]}</b>",
    offsetgroup=0,
    marker=dict(color=CORES[1])
))

fig.show(config={"displayModeBar": False})